In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataframeExercise").getOrCreate()

In [2]:
spark.sparkContext

<SparkContext master=local[*] appName=DataframeExercise>

In [3]:

from pyspark.sql import SQLContext


from pyspark.sql.functions import split
from pyspark.sql.functions import col

from pyspark.sql.functions import regexp_replace

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("app_analytics").getOrCreate()
spark

In [43]:
df = spark.read.format("csv").option("inferSchema", 'true').option("header", 'true').option('sep',",").load("churn.csv")

In [44]:
df.show(2)

+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+
|state|account length|area code|phone number|international plan|voice mail plan|number vmail messages|total day minutes|total day calls|total day charge|total eve minutes|total eve calls|total eve charge|total night minutes|total night calls|total night charge|total intl minutes|total intl calls|total intl charge|customer service calls|churn|
+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-------------

In [45]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

TypeError: 'str' object is not callable

In [46]:
df.createOrReplaceTempView("df_sql")

In [47]:
df.columns

['state',
 'account length',
 'area code',
 'phone number',
 'international plan',
 'voice mail plan',
 'number vmail messages',
 'total day minutes',
 'total day calls',
 'total day charge',
 'total eve minutes',
 'total eve calls',
 'total eve charge',
 'total night minutes',
 'total night calls',
 'total night charge',
 'total intl minutes',
 'total intl calls',
 'total intl charge',
 'customer service calls',
 'churn']

In [48]:
df1 = df
for col in df.columns:
 df1 = df1.withColumnRenamed(col,col.replace(" ", "_"))

In [49]:
df1.show(3)

+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+
|state|account_length|area_code|phone_number|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|customer_service_calls|churn|
+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-------------

In [50]:
df1.show(1)

+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+
|state|account_length|area_code|phone_number|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|customer_service_calls|churn|
+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-------------

In [51]:
df1.createOrReplaceTempView('df_sql')

In [52]:
df2 = spark.sql("select *,case when churn = 'false' then 0 else 1 end as y from df_sql")

In [53]:
df2.createOrReplaceTempView('df_sql')

In [54]:
spark.sql("select y,count(*) from df_sql group by y").show()

+---+--------+
|  y|count(1)|
+---+--------+
|  1|     483|
|  0|    2850|
+---+--------+



In [55]:
continuous_features = [ 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'customer_service_calls'
 'y']

In [56]:
categorical_features =['state',
 'account_length',
 'area_code',
 'international_plan',
 'voice_mail_plan',
]

In [57]:
#Encoding all categorical features
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer

In [58]:
# create object of StringIndexer class and specify input and output column
SI_state = StringIndexer(inputCol='state',outputCol='state_Index')
SI_account_length = StringIndexer(inputCol='account_length',outputCol='account_length_Index')
SI_area_code = StringIndexer(inputCol='area_code',outputCol='area_code_Index')
SI_international_plan = StringIndexer(inputCol='international_plan',outputCol='international_plan_Index')
SI_voice_mail_plan = StringIndexer(inputCol='voice_mail_plan',outputCol='voice_mail_plan_Index')

In [59]:

# transform the data
df2 = SI_state.fit(df2).transform(df2)
df2 = SI_account_length.fit(df2).transform(df2)
df2 = SI_area_code.fit(df2).transform(df2)
df2 = SI_international_plan.fit(df2).transform(df2)
df2 = SI_voice_mail_plan.fit(df2).transform(df2)


# view the transformed data
df2.select('state', 'state_Index', 'account_length', 'account_length_Index', 'area_code', 'area_code_Index','international_plan','international_plan_Index','voice_mail_plan','voice_mail_plan_Index').show(10)

+-----+-----------+--------------+--------------------+---------+---------------+------------------+------------------------+---------------+---------------------+
|state|state_Index|account_length|account_length_Index|area_code|area_code_Index|international_plan|international_plan_Index|voice_mail_plan|voice_mail_plan_Index|
+-----+-----------+--------------+--------------------+---------+---------------+------------------+------------------------+---------------+---------------------+
|   KS|       17.0|           128|                67.0|      415|            0.0|                no|                     0.0|            yes|                  1.0|
|   OH|        5.0|           107|                 9.0|      415|            0.0|                no|                     0.0|            yes|                  1.0|
|   NJ|       18.0|           137|                55.0|      415|            0.0|                no|                     0.0|             no|                  0.0|
|   OH|        5

In [66]:
df2.columns

['state',
 'account_length',
 'area_code',
 'phone_number',
 'international_plan',
 'voice_mail_plan',
 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'customer_service_calls',
 'churn',
 'y',
 'state_Index',
 'account_length_Index',
 'area_code_Index',
 'international_plan_Index',
 'voice_mail_plan_Index']

In [70]:
assesmble=VectorAssembler(inputCols=[
 'account_length',
 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'customer_service_calls',
 'state_Index',
 'account_length_Index',
 'area_code_Index',
 'international_plan_Index',
 'voice_mail_plan_Index'],outputCol='features')


In [67]:
#the transform method to  transform our data 
df3=assesmble.transform(df2)

In [71]:
df3.show()

+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+---+-----------+--------------------+---------------+------------------------+---------------------+--------------------+
|state|account_length|area_code|phone_number|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|customer_service_calls|churn|  y|state_Index|account_length_Index|area_code_Index|international_plan_Index|voice_mail_plan_Index|            features|
+-----+--------------+---------+------------+------------------+----

In [73]:
df4=df3.select('features','y')

In [74]:
df4.show()

+--------------------+---+
|            features|  y|
+--------------------+---+
|[128.0,25.0,265.1...|  0|
|[107.0,26.0,161.6...|  0|
|[137.0,0.0,243.4,...|  0|
|[84.0,0.0,299.4,7...|  0|
|[75.0,0.0,166.7,1...|  0|
|[118.0,0.0,223.4,...|  0|
|[121.0,24.0,218.2...|  0|
|[147.0,0.0,157.0,...|  0|
|[117.0,0.0,184.5,...|  0|
|[141.0,37.0,258.6...|  0|
|[65.0,0.0,129.1,1...|  1|
|[74.0,0.0,187.7,1...|  0|
|[168.0,0.0,128.8,...|  0|
|[95.0,0.0,156.6,8...|  0|
|[62.0,0.0,120.7,7...|  0|
|[161.0,0.0,332.9,...|  1|
|[85.0,27.0,196.4,...|  0|
|[93.0,0.0,190.7,1...|  0|
|[76.0,33.0,189.7,...|  0|
|[73.0,0.0,224.4,9...|  0|
+--------------------+---+
only showing top 20 rows



In [76]:
(train, test) = df4.randomSplit([.7,.3])

In [77]:
train.show()

+--------------------+---+
|            features|  y|
+--------------------+---+
|(20,[0,2,3,4,5,6,...|  0|
|[1.0,0.0,123.8,11...|  0|
|[1.0,0.0,144.8,10...|  0|
|[1.0,0.0,175.2,74...|  0|
|[1.0,0.0,182.1,10...|  1|
|[1.0,26.0,146.6,6...|  0|
|[1.0,26.0,208.0,1...|  0|
|[1.0,30.0,183.1,9...|  0|
|[3.0,0.0,139.0,99...|  0|
|[3.0,0.0,161.0,96...|  0|
|[3.0,0.0,185.0,12...|  0|
|[3.0,27.0,67.4,11...|  0|
|[3.0,36.0,118.1,1...|  0|
|[4.0,0.0,145.3,89...|  0|
|[6.0,0.0,183.6,11...|  0|
|[6.0,0.0,226.5,93...|  0|
|[7.0,0.0,206.7,87...|  0|
|[7.0,30.0,221.4,1...|  0|
|[9.0,16.0,88.5,87...|  0|
|[9.0,31.0,193.8,1...|  0|
+--------------------+---+
only showing top 20 rows



In [78]:
test.show()

+--------------------+---+
|            features|  y|
+--------------------+---+
|[1.0,0.0,196.1,10...|  0|
|[2.0,0.0,132.1,42...|  1|
|[5.0,0.0,199.2,10...|  0|
|[8.0,36.0,242.9,6...|  0|
|[10.0,0.0,183.0,1...|  0|
|[10.0,0.0,222.2,1...|  0|
|[12.0,0.0,204.6,9...|  0|
|[12.0,0.0,249.6,1...|  1|
|[13.0,25.0,176.6,...|  0|
|[16.0,0.0,144.8,8...|  0|
|[16.0,0.0,161.9,1...|  0|
|[16.0,0.0,174.7,8...|  0|
|[16.0,0.0,209.5,8...|  0|
|[19.0,0.0,186.1,9...|  0|
|[19.0,0.0,201.5,1...|  0|
|[20.0,35.0,171.5,...|  0|
|[21.0,0.0,92.6,95...|  1|
|[21.0,0.0,146.0,7...|  0|
|[21.0,19.0,132.7,...|  0|
|[22.0,0.0,160.4,1...|  0|
+--------------------+---+
only showing top 20 rows



In [79]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [81]:
model=LogisticRegression(labelCol='y')

In [82]:
model=model.fit(train)

In [83]:
summary=model.summary

In [84]:
summary.predictions.describe().show()

+-------+------------------+--------------------+
|summary|                 y|          prediction|
+-------+------------------+--------------------+
|  count|              2315|                2315|
|   mean| 0.150755939524838|0.058315334773218146|
| stddev|0.3578881306322746| 0.23438939393368008|
|    min|               0.0|                 0.0|
|    max|               1.0|                 1.0|
+-------+------------------+--------------------+



In [85]:
pred=model.evaluate(test)

In [86]:
pred.predictions.show()

+--------------------+---+--------------------+--------------------+----------+
|            features|  y|       rawPrediction|         probability|prediction|
+--------------------+---+--------------------+--------------------+----------+
|[1.0,0.0,196.1,10...|  0|[1.71001826844561...|[0.84683865372118...|       0.0|
|[2.0,0.0,132.1,42...|  1|[1.02153899634942...|[0.73527226887271...|       0.0|
|[5.0,0.0,199.2,10...|  0|[1.00378991051957...|[0.73180306743056...|       0.0|
|[8.0,36.0,242.9,6...|  0|[4.23772349591563...|[0.98576512947834...|       0.0|
|[10.0,0.0,183.0,1...|  0|[2.24841064319680...|[0.90451335225197...|       0.0|
|[10.0,0.0,222.2,1...|  0|[2.15897297475679...|[0.89650429547594...|       0.0|
|[12.0,0.0,204.6,9...|  0|[2.14924338408304...|[0.89559805281508...|       0.0|
|[12.0,0.0,249.6,1...|  1|[0.59739736899082...|[0.64506063979244...|       0.0|
|[13.0,25.0,176.6,...|  0|[3.72988155373243...|[0.97656662192475...|       0.0|
|[16.0,0.0,144.8,8...|  0|[2.96270968087

In [91]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='y')

In [99]:
#evaluator.evaluate(predictions.predictions)

In [93]:
pred1=model.transform(test)

In [96]:
 eval_accuracy = (MulticlassClassificationEvaluator(labelCol="y", predictionCol="prediction", metricName="accuracy"))

In [97]:
 accuracy = eval_accuracy.evaluate(pred1)

In [98]:
accuracy

0.8713163064833006

In [100]:
Error = 1-accuracy

In [101]:
Error

0.1286836935166994

In [104]:
eval_precision = (MulticlassClassificationEvaluator(labelCol="y", predictionCol="prediction", metricName="weightedPrecision"))

In [105]:
precision =  eval_precision.evaluate(pred1)

In [106]:
precision

0.8419697946244321

In [109]:
eval_recall=(MulticlassClassificationEvaluator(labelCol="y", predictionCol="prediction", metricName="weightedRecall"))


In [110]:
recall =eval_recall.evaluate(pred1)

In [111]:
recall

0.8713163064833006

In [114]:
eval_f1 =(MulticlassClassificationEvaluator(labelCol="y", predictionCol="prediction", metricName="f1"))

In [115]:
f1 =  eval_f1.evaluate(pred1)

In [116]:
f1

0.845334552614534